In [1]:
from google.colab import drive
import os
drive.mount('/content/gdrive')
os.chdir("gdrive/MyDrive/aYin/alaska")

Mounted at /content/gdrive


In [2]:
### Import Natural Language Toolkit (NLTK) libraries for text processing
from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import SnowballStemmer

import string
import re
import pandas as pd

import sys
import string
import re

### Utilize scikit-learn libraries
### Add additional libraries for performance evaluation and visualization
from sklearn import model_selection, preprocessing, linear_model, naive_bayes, metrics, svm
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn import decomposition, ensemble
from sklearn.metrics import f1_score
from sklearn.metrics import classification_report

import numpy, string
import pandas

from matplotlib import pyplot as plt
import numpy as np

from nltk.corpus import stopwords
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.stem import SnowballStemmer

from nltk.tokenize import TreebankWordTokenizer
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
from sklearn.naive_bayes import MultinomialNB
from sklearn.svm import SVC

In [3]:
### Function for text preprocessing:
### Text cleaning by removing stop words, digits, and non-ASCII characters
### Keep stems
def stem_prerpare(pathO, pathD):
	print("program starts ...")
	stop = set(stopwords.words('english'))
	exclude = set(string.punctuation)
	lemma = WordNetLemmatizer()
	mFile = open(pathO,'r',encoding='utf-8')
	oFile = open(pathD,'w',encoding='utf-8')
	next(mFile)
	for mRow in mFile:
		data = mRow.rstrip().split('\t')
		# title = data[0]
		content = data[-1]
		text = content.rstrip()
		text = text.lower().split()
		text = [w for w in text if not w in stop and len(w) >= 3]
		text = [ch for ch in text if ch not in exclude]
		text = " ".join(text)
		text = re.sub(r"[^a-zA-Z]", " ", text)
		text = re.sub(r"what\'s", "what is ", text)
		text = re.sub(r"\'s", " ", text)
		text = re.sub(r"\'ve", " have ", text)
		text = re.sub(r"n\'t", " not ", text)
		text = re.sub(r"i\'m", "i am ", text)
		text = re.sub(r"\'re", " are ", text)
		text = re.sub(r"\'d", " would ", text)
		text = re.sub(r"\'ll", " will ", text)
		text = re.sub(r",", " ", text)
		text = re.sub(r"\.", " ", text)
		text = re.sub(r"!", " ! ", text)
		text = re.sub(r"\/", " ", text)
		text = re.sub(r"\^", " ^ ", text)
		text = re.sub(r"\+", " + ", text)
		text = re.sub(r"\-", " - ", text)
		text = re.sub(r"\=", " = ", text)
		text = re.sub(r":", " : ", text)
		text = re.sub(r"\s{2,}", " ", text)
		text = text.split()
		stemmer = SnowballStemmer('english')
		stemmed_words = [stemmer.stem(word) for word in text]
		text = " ".join(stemmed_words)
		mString = '\t'.join(data[:-1]) + '\t' + text + '\n'
		oFile.write(mString)
	oFile.close()
	mFile.close()

### Save cleaned data to a file

In [4]:
### Function for training the model for the classifier
def train_model(classifier, feature_vector_train, label, feature_vector_valid, valid_y, is_neural_net=False):
	classifier.fit(feature_vector_train, label)
	predictions = classifier.predict(feature_vector_valid)
	if is_neural_net:
		predictions = predictions.argmax(axis=-1)
	return [metrics.recall_score(valid_y, predictions, average='weighted'), metrics.accuracy_score(valid_y, predictions), metrics.f1_score(valid_y, predictions, average='weighted')]


In [5]:
def classify(file1, file2):
	print("program starts")
	mFile = open('alaska_climate_change_lables_stems.tsv','r', encoding='utf-8')
	labels, texts = [], []
	for mRow in mFile:
		content = mRow.rstrip().split('\t')
		labels.append(content[-2].lower())
		# texts.append(" ".join(content[-1]))
		texts.append(content[-1])
	# print(labels)
	print(texts)

	trainDF = pandas.DataFrame()
	trainDF['text'] = texts
	trainDF['label'] = labels
	train_x, valid_x, train_y, valid_y = model_selection.train_test_split(trainDF['text'], trainDF['label'], test_size=0.2, shuffle=True, random_state=0)

	count_vect = CountVectorizer()
	tokenizer = TreebankWordTokenizer()
	count_vect.set_params(tokenizer=tokenizer.tokenize)
	count_vect.set_params(stop_words='english')
	count_vect.set_params(ngram_range=(1,2))
	count_vect.set_params(max_df=0.5)
	count_vect.set_params(min_df=2)

	count_vect.fit(trainDF['text'])

	xtrain_count =  count_vect.transform(train_x)
	xvalid_count =  count_vect.transform(valid_x)

	encoder = preprocessing.LabelEncoder()
	train_y = encoder.fit_transform(train_y)
	valid_y = encoder.fit_transform(valid_y)

	### Print the information for accuracy
	accuracy = train_model(ensemble.RandomForestClassifier(), xtrain_count, train_y, xvalid_count,valid_y)
	print("Random Forest, Count Vectors recall: ", accuracy[0])
	print("Random Forest, Count Vectors precision: ", accuracy[1])
	print("Random Forest, Count Vectors F1 score: ", accuracy[2])


	accuracy_nb = train_model(MultinomialNB(), xtrain_count, train_y, xvalid_count, valid_y)
	print("Naive Bayes, Count Vectors recall: ", accuracy_nb[0])
	print("Naive Bayes, Count Vectors precision: ", accuracy_nb[1])
	print("Naive Bayes, Count Vectors F1 score: ", accuracy_nb[2])

	accuracy_svm = train_model(SVC(kernel='linear'), xtrain_count, train_y, xvalid_count, valid_y)
	print("SVM, Count Vectors recall: ", accuracy_svm[0])
	print("SVM, Count Vectors precision: ", accuracy_svm[1])
	print("SVM, Count Vectors F1 score: ", accuracy_svm[2])

	# clf = ensemble.RandomForestClassifier(random_state=0).fit(xtrain_count,train_y)

	# pFile = open(file1,'r',encoding='utf-8')
	# oFile = open(file2,'w',encoding='utf-8')

	# for mRow in pFile:
	# 	texts = []
	# 	content = mRow.strip().split("\t")
	# 	texts.append(" ".join(content[-1].split()))
	# 	xx = count_vect.transform(texts)
	# 	results = clf.predict(xx)
	# 	mString = mRow.strip() + "\t" + str(results[0]) + "\n"
	# 	oFile.write(mString)
	# oFile.close()


In [7]:
if __name__ == '__main__':
	# stem_prerpare('alaska_climate_change_lables.tsv', 'alaska_climate_change_lables_stems.tsv')
	# ### Perform classification and save the data to a new file
	classify('alaska_climate_change_lables_stems.tsv', 'alaska_climate_change_lables_stems_class.csv')

program starts
['zero shishmaref rain wtf globalwarmingmyass keep loos villag', 'tribal relat tell stori work w alaska tribe climat chang', 'gotta adjust climat hell', 'c everi varieti bamboo around world flower time regardless locat climat known factor', 'everyth man made natur scienc trust shit come evolut global warm', 'ooof i m acclim weather now i m pretti sure would melt', 'ecologist warn global ecosystem collaps abrupt climat chang decad flood', 'global ecolog system collaps die destroy ecosystem habitat', 'climat chang could lead global conflict', 'believ evolut global warm particip anyth use scientif process', 'dont know thise fire ukrain caus global warm putin strong arm ukrain typic russian way', 'ohh whuddupp global warm', 'team pelican ts kotzebu discuss eros invas speci climat chang', 'acclim long run stickingtotheplan', 'dude fairbank i m rock short sleev guess that s happen get acclim negat', 'assum refer poor climat outlook catch freez s us listen', 'joe s rant climat 

/usr/local/lib/python3.10/dist-packages/sklearn/feature_extraction/text.py:521: UserWarning: The parameter 'token_pattern' will not be used since 'tokenizer' is not None'
  warnings.warn(


Random Forest, Count Vectors recall:  0.849438202247191
Random Forest, Count Vectors precision:  0.849438202247191
Random Forest, Count Vectors F1 score:  0.8412111759595909
Naive Bayes, Count Vectors recall:  0.7573033707865169
Naive Bayes, Count Vectors precision:  0.7573033707865169
Naive Bayes, Count Vectors F1 score:  0.7534762316335349
SVM, Count Vectors recall:  0.7865168539325843
SVM, Count Vectors precision:  0.7865168539325843
SVM, Count Vectors F1 score:  0.7834540111308576
